In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import torch

#Problem 1

Write a web scraping script which takes an input of any film actor and gives the output of filmography of that actor in descending order.

Use web scraping method


In [ ]:
! pip install IMDbPY

In [23]:
import imdb
ia = imdb.IMDb() 
name = "inception" 
# Here the input is an movie of an actor, the acotr has acted
# For Leonardo Dicarpio, its Inception
search = ia.search_movie(name) 
movie_name = search[0]
movie_ID = search[0].getID() # We get the movie ID from the iMDB database
movie = ia.get_movie(movie_ID)
print('Movie:', movie_name)
print('ID:', movie_ID)

Movie: Inception
ID: 1375666


In [27]:
actor = movie['cast'][0]
actor_ID = actor.getID()  # Taking out the actor ID 
print("Actor:", actor)
print("ID:", actor_ID)

Actor: Leonardo DiCaprio
ID: 0000138


In [39]:
# Actor filmography
actor_results = ia.get_person_filmography(actor_ID)
# printing movie name
# Filmography of Leonardo DiCarpio in descending order
actor_results

{'data': {'akas': ['Leonardo Di Caprio', 'Leo DiCaprio'],
  'birth info': {'birth place': 'Hollywood, Los Angeles, California, USA'},
  'filmography': {'actor': [<Movie id:7428530[http] title:_Roosevelt (2023)_>,
    <Movie id:15909214[http] title:_Jim Jones () (None)_>,
    <Movie id:5537002[http] title:_Killers of the Flower Moon (2022)_>,
    <Movie id:11286314[http] title:_Don't Look Up (2021)_>,
    <Movie id:7131622[http] title:_Once Upon a Time... In Hollywood (2019)_>,
    <Movie id:1663202[http] title:_The Revenant (2015)_>,
    <Movie id:4016250[http] title:_The Audition (III) (2015)_>,
    <Movie id:0993846[http] title:_The Wolf of Wall Street (2013)_>,
    <Movie id:1343092[http] title:_The Great Gatsby (2013)_>,
    <Movie id:15739482[http] title:_Titanic: Deleted Scenes (2012)_>,
    <Movie id:1853728[http] title:_Django Unchained (2012)_>,
    <Movie id:1616195[http] title:_J. Edgar (2011)_>,
    <Movie id:1790736[http] title:_Inception: Motion Comics (2010)_>,
    <Movi

#Problem 2

Create an ML algorithm to classify the planets as Candidate/False positive/Confirmed etc based on the  column “koi_disposition”.


## Loading and preparing the data

In [174]:
data = pd.read_csv("/content/cumulative.csv")
data.head(2)

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347


In [175]:
print("No of rows:", data.shape[0])
print("No of columns:", data.shape[1])

No of rows: 9564
No of columns: 50


In [176]:
# Checking for missing values
data.isnull().sum()

rowid                   0
kepid                   0
kepoi_name              0
kepler_name          7270
koi_disposition         0
koi_pdisposition        0
koi_score            1510
koi_fpflag_nt           0
koi_fpflag_ss           0
koi_fpflag_co           0
koi_fpflag_ec           0
koi_period              0
koi_period_err1       454
koi_period_err2       454
koi_time0bk             0
koi_time0bk_err1      454
koi_time0bk_err2      454
koi_impact            363
koi_impact_err1       454
koi_impact_err2       454
koi_duration            0
koi_duration_err1     454
koi_duration_err2     454
koi_depth             363
koi_depth_err1        454
koi_depth_err2        454
koi_prad              363
koi_prad_err1         363
koi_prad_err2         363
koi_teq               363
koi_teq_err1         9564
koi_teq_err2         9564
koi_insol             321
koi_insol_err1        321
koi_insol_err2        321
koi_model_snr         363
koi_tce_plnt_num      346
koi_tce_delivname     346
koi_steff   

In [177]:
# Target column
data['koi_disposition'].value_counts()

FALSE POSITIVE    5023
CONFIRMED         2293
CANDIDATE         2248
Name: koi_disposition, dtype: int64

In [178]:
data['koi_pdisposition'].value_counts()

FALSE POSITIVE    5068
CANDIDATE         4496
Name: koi_pdisposition, dtype: int64

In [179]:
# dropping redundant columns
data = data.drop(['rowid', 'kepid', 'kepoi_name', 'kepler_name',  'koi_pdisposition' , 'koi_teq_err1', 'koi_teq_err2'], axis = 1)
print(data.shape)
print()
data.isnull().sum()

(9564, 43)



koi_disposition         0
koi_score            1510
koi_fpflag_nt           0
koi_fpflag_ss           0
koi_fpflag_co           0
koi_fpflag_ec           0
koi_period              0
koi_period_err1       454
koi_period_err2       454
koi_time0bk             0
koi_time0bk_err1      454
koi_time0bk_err2      454
koi_impact            363
koi_impact_err1       454
koi_impact_err2       454
koi_duration            0
koi_duration_err1     454
koi_duration_err2     454
koi_depth             363
koi_depth_err1        454
koi_depth_err2        454
koi_prad              363
koi_prad_err1         363
koi_prad_err2         363
koi_teq               363
koi_insol             321
koi_insol_err1        321
koi_insol_err2        321
koi_model_snr         363
koi_tce_plnt_num      346
koi_tce_delivname     346
koi_steff             363
koi_steff_err1        468
koi_steff_err2        483
koi_slogg             363
koi_slogg_err1        468
koi_slogg_err2        468
koi_srad              363
koi_srad_err

In [180]:
# Dropping the Null values
data = data.dropna()
data.shape

(7803, 43)

In [181]:
# Datatypes 
data.dtypes

koi_disposition       object
koi_score            float64
koi_fpflag_nt          int64
koi_fpflag_ss          int64
koi_fpflag_co          int64
koi_fpflag_ec          int64
koi_period           float64
koi_period_err1      float64
koi_period_err2      float64
koi_time0bk          float64
koi_time0bk_err1     float64
koi_time0bk_err2     float64
koi_impact           float64
koi_impact_err1      float64
koi_impact_err2      float64
koi_duration         float64
koi_duration_err1    float64
koi_duration_err2    float64
koi_depth            float64
koi_depth_err1       float64
koi_depth_err2       float64
koi_prad             float64
koi_prad_err1        float64
koi_prad_err2        float64
koi_teq              float64
koi_insol            float64
koi_insol_err1       float64
koi_insol_err2       float64
koi_model_snr        float64
koi_tce_plnt_num     float64
koi_tce_delivname     object
koi_steff            float64
koi_steff_err1       float64
koi_steff_err2       float64
koi_slogg     

In [182]:
data['koi_tce_delivname'].value_counts()

q1_q17_dr25_tce    7803
Name: koi_tce_delivname, dtype: int64

In [183]:
# Column koi_tce_delivname has same value across all the rows so this column is redundant
data = data.drop("koi_tce_delivname", axis = 1)
data.shape

(7803, 42)

In [184]:
data.isnull().sum()

koi_disposition      0
koi_score            0
koi_fpflag_nt        0
koi_fpflag_ss        0
koi_fpflag_co        0
koi_fpflag_ec        0
koi_period           0
koi_period_err1      0
koi_period_err2      0
koi_time0bk          0
koi_time0bk_err1     0
koi_time0bk_err2     0
koi_impact           0
koi_impact_err1      0
koi_impact_err2      0
koi_duration         0
koi_duration_err1    0
koi_duration_err2    0
koi_depth            0
koi_depth_err1       0
koi_depth_err2       0
koi_prad             0
koi_prad_err1        0
koi_prad_err2        0
koi_teq              0
koi_insol            0
koi_insol_err1       0
koi_insol_err2       0
koi_model_snr        0
koi_tce_plnt_num     0
koi_steff            0
koi_steff_err1       0
koi_steff_err2       0
koi_slogg            0
koi_slogg_err1       0
koi_slogg_err2       0
koi_srad             0
koi_srad_err1        0
koi_srad_err2        0
ra                   0
dec                  0
koi_kepmag           0
dtype: int64

## Feature selection

In [185]:
y = data['koi_disposition']
x = data.drop('koi_disposition', axis = 1)

In [186]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test = train_test_split(x,y, test_size=0.3, random_state = 0)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5462, 41)
(2341, 41)
(5462,)
(2341,)


In [187]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
rf = RandomForestClassifier(random_state = 0, n_estimators = 100)
rf.fit(X_train, y_train)
imp_feat = list(zip(x.columns, rf.feature_importances_))
imp_feat

[('koi_score', 0.24241465002507723),
 ('koi_fpflag_nt', 0.03117695326695575),
 ('koi_fpflag_ss', 0.0479062101696031),
 ('koi_fpflag_co', 0.07650444756673015),
 ('koi_fpflag_ec', 0.019228864843063256),
 ('koi_period', 0.014362880860995388),
 ('koi_period_err1', 0.011604040339952455),
 ('koi_period_err2', 0.01054986926108522),
 ('koi_time0bk', 0.011173647794868203),
 ('koi_time0bk_err1', 0.022708880995145894),
 ('koi_time0bk_err2', 0.01964803031026614),
 ('koi_impact', 0.019133533310725877),
 ('koi_impact_err1', 0.009538692076058291),
 ('koi_impact_err2', 0.009442670175399822),
 ('koi_duration', 0.019639441286486946),
 ('koi_duration_err1', 0.021336230070382465),
 ('koi_duration_err2', 0.027416283709405537),
 ('koi_depth', 0.016619512021005787),
 ('koi_depth_err1', 0.011176133949374237),
 ('koi_depth_err2', 0.01087711589230971),
 ('koi_prad', 0.03998049840805436),
 ('koi_prad_err1', 0.029881480887202425),
 ('koi_prad_err2', 0.0367271747637725),
 ('koi_teq', 0.012507006376046445),
 ('koi_

In [188]:
sel = SelectFromModel(RandomForestClassifier(random_state = 0, n_estimators = 100), threshold = 0.01)
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(random_state=0),
                threshold=0.01)

In [189]:
selected_feat= X_train.columns[(sel.get_support())]
print(len(selected_feat))

28


In [190]:
print("Selected Features:\n", list(selected_feat))

Selected Features:
 ['koi_score', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2', 'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact', 'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1', 'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2', 'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2', 'koi_model_snr', 'koi_steff_err1', 'koi_steff_err2']


In [191]:
x_new = x[list(selected_feat)]
print(x_new.shape)

(7803, 28)


In [192]:
X_trainn,X_testn,y_trainn, y_testn = train_test_split(x_new,y, test_size=0.3, random_state = 0)
print(X_trainn.shape)
print(X_testn.shape)
print(y_trainn.shape)
print(y_testn.shape)

(5462, 28)
(2341, 28)
(5462,)
(2341,)


## Training the models

## Logistic Regression

In [193]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=0)

In [194]:
y_pred = lr.predict(X_test)
from sklearn.metrics import classification_report
cr = classification_report(y_test, y_pred, digits = 4)
print(cr)

                precision    recall  f1-score   support

     CANDIDATE     0.0000    0.0000    0.0000       508
     CONFIRMED     0.4895    0.8374    0.6178       695
FALSE POSITIVE     0.7830    0.7926    0.7878      1138

      accuracy                         0.6339      2341
     macro avg     0.4242    0.5433    0.4685      2341
  weighted avg     0.5259    0.6339    0.5664      2341



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Decision Tree

In [195]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 0, max_leaf_nodes = 6)
dt.fit(X_train, y_train)

DecisionTreeClassifier(max_leaf_nodes=6, random_state=0)

In [196]:
y_pred = dt.predict(X_test)
from sklearn.metrics import classification_report
cr = classification_report(y_test, y_pred, digits = 4)
print(cr)

                precision    recall  f1-score   support

     CANDIDATE     0.6992    0.6496    0.6735       508
     CONFIRMED     0.7614    0.7942    0.7775       695
FALSE POSITIVE     0.9781    0.9833    0.9807      1138

      accuracy                         0.8548      2341
     macro avg     0.8129    0.8091    0.8106      2341
  weighted avg     0.8533    0.8548    0.8537      2341



## Random Forest

In [197]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 0, n_estimators = 500, max_leaf_nodes = 6)
rf.fit(X_train, y_train)

RandomForestClassifier(max_leaf_nodes=6, n_estimators=500, random_state=0)

In [198]:
y_pred = rf.predict(X_test)
from sklearn.metrics import classification_report
cr = classification_report(y_test, y_pred, digits = 4)
print(cr)

                precision    recall  f1-score   support

     CANDIDATE     0.7324    0.5335    0.6173       508
     CONFIRMED     0.7526    0.8317    0.7902       695
FALSE POSITIVE     0.9418    0.9956    0.9680      1138

      accuracy                         0.8466      2341
     macro avg     0.8089    0.7869    0.7918      2341
  weighted avg     0.8402    0.8466    0.8391      2341



## XGBoost

In [199]:
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state = 0, n_estimators = 600, max_leaf_nodes = 5)
xgb.fit(X_train, y_train)

XGBClassifier(max_leaf_nodes=5, n_estimators=600, objective='multi:softprob')

In [200]:
y_pred = xgb.predict(X_test)
from sklearn.metrics import classification_report
cr = classification_report(y_test, y_pred, digits = 4)
print(cr)

                precision    recall  f1-score   support

     CANDIDATE     0.7663    0.7165    0.7406       508
     CONFIRMED     0.7981    0.8302    0.8138       695
FALSE POSITIVE     0.9878    0.9921    0.9899      1138

      accuracy                         0.8842      2341
     macro avg     0.8507    0.8463    0.8481      2341
  weighted avg     0.8834    0.8842    0.8835      2341



# Sub-questions for Problem 2

1. Why did you choose the particular algorithm?

Ans: I have tried various classical machine learning algorithms. XGBoost gave the highest accuracy so this is the best model. This is an boosting algorithm which reduces the overfitting.

2. What are the different tuning methods used for the algorithm?

Ans: I increased the number of trees and also tried with different values of max_leaf_nodes.

3. Did you consider any other choice of algorithm?Why or why not?

Ans: I have tried various other classical machine learning algorithms such as:

1. Logistic Regression

2. Decision Tree

3. Random Forest

4. XGBoost

4. What is the accuracy?

Ans: I have achieved highest accuracy with XGBoost i.e 88%.


5. What are the different types of metrics that can be used to evaluate the model?

Ans: The different types of metrics that can be used to evaluate the models are:

1. Accuracy: Accuracy is the number of correctly predicted data points out of all the data points.

2. Precision:  Precision refers to the number of true positives divided by the total number of positive predictions (i.e., the number of true positives plus the number of false positives).

3. Recall: Mathematically, we define recall as the number of true positives divided by the number of true positives plus the number of false negatives.

4. F1-score:  F1 score is defined as the harmonic mean between precision and recall. F1 score is used when dealing with imbalanced data as accuracy is not the preferred metric in these cases.